In [37]:
import torch

In [38]:
score = torch.load('test/uncertainty_scores.pt')
label = torch.load('test/uncertainty_labels.pt')

In [39]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import *


In [40]:
print(score.shape, label.shape)

torch.Size([64, 200, 200]) torch.Size([64, 200, 200])


In [41]:
cohen_kappa_score(label.flatten(), score.flatten() > .5)

0.35456385404955126

In [51]:
patch_acc = []
patch_unc = []

anchor = (0, 0)
window_size = 1
last_anchor = (label.shape[1] - window_size, label.shape[2] - window_size)

while anchor != last_anchor:
    label_window = label[:, anchor[0]:anchor[0] + window_size, anchor[1]:anchor[1] + window_size]
    uncertainty_window = score[:, anchor[0]:anchor[0] + window_size, anchor[1]:anchor[1] + window_size]

    accuracy = torch.sum(label_window, dim=(1, 2)) / (window_size ** 2)
    avg_uncertainty = torch.mean(uncertainty_window, dim=(1, 2))
    accurate = accuracy > .5
    uncertain = avg_uncertainty >= .4

    patch_acc += accurate.flatten()
    patch_unc += avg_uncertainty.flatten()

    if anchor[1] < label.shape[1] - window_size:
        anchor = (anchor[0], anchor[1] + window_size)
    else:
        anchor = (anchor[0] + window_size, 0)

In [49]:
import numpy as np

In [53]:
cohen_kappa_score(np.array(patch_acc), np.array(patch_unc) > .4)

0.40265468427868556